<a href="https://colab.research.google.com/github/komal83/timeofyourhatepaper/blob/master/gpu_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# @title Preparation
!pip install -q keras-bert keras-rectified-adam
!wget -q https://storage.googleapis.com/albert_files/alberto_tweets_uncased_L-12_H-768_A-12.zip
!unzip -o alberto_tweets_uncased_L-12_H-768_A-12.zip

KeyboardInterrupt: ignored

In [0]:
# @title Import and drive mount

import os
import tensorflow as tf
import keras
from keras_radam import RAdam
from keras_bert import get_custom_objects
import numpy as np
from tqdm import tqdm
from keras_bert import Tokenizer
import pandas as pd
import tensorflow.keras.backend as K
import sys
from sklearn.metrics import classification_report
from google.colab import drive

Using TensorFlow backend.


In [0]:
# @title Constants

np.random.seed(999)
SEQ_LEN = 128
BATCH_SIZE = 8
EPOCHS = 5
LR = 1e-5

In [0]:
# @title Environment

pretrained_path = 'albert_tweets_uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'model.ckpt-1000000')
vocab_path = os.path.join(pretrained_path, 'vocabulary_lower_case_128.txt')

In [0]:
# @title Load Basic Model
import codecs
from keras_bert import load_trained_model_from_checkpoint

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)


model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=True,
    trainable=True,
    seq_len=SEQ_LEN,
)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









In [0]:
# @title Convert Data to Array


tokenizer = Tokenizer(token_dict)


def load_data(path):
    global tokenizer
    indices, sentiments = [], []
    #data= pd.read_csv(path,names=['tweet_id', 'text', 'label'],sep='\t')
    data= pd.read_csv(path,names=['tweet_id', 'text', 'label'])
    for text in tqdm(data.text):
        '''for folder, sentiment in (('neg', 0), ('pos', 1)):
        folder = os.path.join(path, folder)
        for name in tqdm(os.listdir(folder)):
            with open(os.path.join(folder, name), 'r') as reader:
                  text = reader.read()'''     
        
        ids, segments = tokenizer.encode(str(text).strip(), max_len=SEQ_LEN)
        indices.append(ids)
    #for sentiment in tqdm(data.label.replace({'no': 0, 'yes': 1})):
    for sentiment in tqdm(data.label):   
        #sentiments.append(sentiment)
        #sentiments.append(eval(sentiment))
        try:
            sentiments.append(int(sentiment))
        except:
            print (sentiment)
            print (type(sentiment))
            sys.exit(0)
    
    items = list(zip(indices, sentiments))
    np.random.shuffle(items)
    indices, sentiments = zip(*items)
    indices = np.array(indices)
    mod = indices.shape[0] % BATCH_SIZE
    if mod > 0:
        indices, sentiments = indices[:-mod], sentiments[:-mod]
    return [indices, np.zeros_like(indices)], np.array(sentiments)

def load_data_train(path):
    global tokenizer
    indices, sentiments = [], []
    #data= pd.read_csv(path,names=['tweet_id', 'text', 'label'],sep='\t')
    data= pd.read_csv(path,names=['tweet_id', 'text', 'label'], quotechar = '"',sep=',')
    for text in tqdm(data.text):
        '''for folder, sentiment in (('neg', 0), ('pos', 1)):
        folder = os.path.join(path, folder)
        for name in tqdm(os.listdir(folder)):
            with open(os.path.join(folder, name), 'r') as reader:
                  text = reader.read()'''     
        
        ids, segments = tokenizer.encode(str(text).strip(), max_len=SEQ_LEN)
        indices.append(ids)
    #for sentiment in tqdm(data.label.replace({'no': 0, 'yes': 1})):
    for sentiment in tqdm(data.label):   
        #sentiments.append(sentiment)
        #sentiments.append(eval(sentiment))
        try:
            sentiments.append(int(sentiment))
        except:
            print (sentiment)
            print (type(sentiment))
            sys.exit(0)
    
    items = list(zip(indices, sentiments))
    np.random.shuffle(items)
    indices, sentiments = zip(*items)
    indices = np.array(indices)
    mod = indices.shape[0] % BATCH_SIZE
    if mod > 0:
        indices, sentiments = indices[:-mod], sentiments[:-mod]
    return [indices, np.zeros_like(indices)], np.array(sentiments)

In [0]:
# @title  Load Data
drive.mount('/content/drive', force_remount=True)
train_x, train_y = load_data('/content/drive/My Drive/IJCai2020/IJCAi2020-dati/training_new.csv')
test_x, test_y = load_data('/content/drive/My Drive/IJCai2020/IJCAi2020-dati/2019_02_k.csv')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


100%|██████████| 2000/2000 [00:00<00:00, 385967.06it/s]


In [0]:

#@title Print Data
#print(train_x[:10],train_y[:10])

In [0]:
# @title Build Custom Model


inputs = model.inputs[:2]
dense = model.get_layer('NSP-Dense').output
outputs = keras.layers.Dense(units=2, activation='softmax')(dense)

model = keras.models.Model(inputs, outputs)
model.compile(
    RAdam(lr=LR),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)

In [0]:
# @title Initialize Variables


sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init_op = tf.variables_initializer(
    [v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables]
)
sess.run(init_op)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# @title Fit

model.fit(
    train_x,
    train_y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
)



Epoch 1/5
11008/11008 [==============================] - 905s 82ms/step - loss: 0.4247 - sparse_categorical_accuracy: 0.8124
Epoch 2/5
11008/11008 [==============================] - 877s 80ms/step - loss: 0.3254 - sparse_categorical_accuracy: 0.8585
Epoch 3/5
11008/11008 [==============================] - 877s 80ms/step - loss: 0.2046 - sparse_categorical_accuracy: 0.9158
Epoch 4/5
11008/11008 [==============================] - 877s 80ms/step - loss: 0.0946 - sparse_categorical_accuracy: 0.9652
Epoch 5/5
11008/11008 [==============================] - 877s 80ms/step - loss: 0.0528 - sparse_categorical_accuracy: 0.9814


In [0]:
# @title Predict

predicts = model.predict(test_x, verbose=True).argmax(axis=-1)
#print(predicts)

NameError: ignored

In [0]:
# @title Classification report
print(classification_report(test_y, predicts))

              precision    recall  f1-score   support

           0       0.96      0.79      0.87      1828
           1       0.23      0.66      0.34       172

    accuracy                           0.78      2000
   macro avg       0.59      0.72      0.60      2000
weighted avg       0.90      0.78      0.82      2000



In [0]:
# @title Data Writing predicted labels to input test file
pred_df = pd.DataFrame(predicts)
data2= pd.read_csv('/content/drive/My Drive/IJCai2020/IJCAi2020-dati/2019_02_k.csv',names=['tweet_id', 'text', 'label'])
data2['predicted'] = pred_df
#print(data2[:10])
data2.to_csv("/content/drive/My Drive/IJCai2020/seeds-media/train_1_test_7_seed_999_NEW_predicted.csv", index=False)

NameError: ignored

In [0]:
#@title Classification Report saving
# Saving metrics from classification report to a csv file incuding seed
clsf_report = pd.DataFrame(classification_report(y_true = test_y, y_pred = predicts, output_dict=True)).transpose()
clsf_report['seed'] = 999

In [0]:
with open('/content/drive/My Drive/IJCai2020/seeds-media/train_1_test_7_clsf_report_NEW_999.csv','a' ) as fd:
    clsf_report.to_csv(fd, index= True)

FileNotFoundError: ignored

In [0]:
#@title Classification Report print

#print(classification_report(test_y, predicts))
print(clsf_report)

              precision    recall  f1-score    support  seed
0              0.961256  0.787199  0.865564  1828.0000   999
1              0.226640  0.662791  0.337778   172.0000   999
accuracy       0.776500  0.776500  0.776500     0.7765   999
macro avg      0.593948  0.724995  0.601671  2000.0000   999
weighted avg   0.898079  0.776500  0.820174  2000.0000   999
